In [2]:
# Edgar new data Jan 2024

import numpy as np
import matplotlib.pyplot as plt
from tqdm import tqdm
import pickle

import torch
import torch.nn.functional as F
from torch.utils.data import DataLoader
from models import *
from utils import *

seed = 2468
np.random.seed(seed)
torch.manual_seed(seed)

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
torch.cuda.get_device_name()

'NVIDIA GeForce RTX 3090'

In [3]:
'''
The imaging is still at 30hz.
The ephys is binarized in 1ms bins.
The speed has corresponding triggers in imaging and ephys.
I.e. for every entry in the speed file, there is a corresponding imaging frame or ephys bin at the given index.
There are files for baseline and for stimulation. In each case, the imaging ROIs and the ephys units are matched.
I.e. index 1 in baseline imaging is the same cell as index 1 is stim imaging.

Please see if this format works for you. Let me know if you want me to downsample all of them to the same frame rate (30Hz) instead if that is easier for you. If this runs ok, then I can run another 1-2 mice through which should put us at 3-4 animals. I will work on getting the combinatorial paradigm in place as well, but it is a little bit complicated right now.
'''
area_ID_ephys = np.load('./data/biological/EB095/area_ID_ephys.npy')        # (106,) 8 LS, 70 ACC
dff_run = np.load('./data/biological/EB095/dff_run.npy')                    # (294, 33897) GCaMP when running
dff_stim = np.load('./data/biological/EB095/dff_stim.npy')                  # (294, 111810) GCaMP when stimulated 
speed_run = np.load('./data/biological/EB095/speed_run.npy')                # (56302,) max 13
speed_triggers_ephys_run = np.load('./data/biological/EB095/speed_triggers_ephys_run.npy')  # (56322)
speed_triggers_img_run = np.load('./data/biological/EB095/speed_triggers_img_run.npy')      # (56322)
spks_run = np.load('./data/biological/EB095/spks_run.npy')                  # (106, 1188000)
spks_stim = np.load('./data/biological/EB095/spks_stim.npy')                # (106, 3799862)
stim_ID = np.load('./data/biological/EB095/stim_ID.npy')                    # (300,), R S or F
stim_triggers_ephys_stim = np.load('./data/biological/EB095/stim_triggers_ephys_stim.npy')  # (300,)
stim_triggers_img_stim = np.load('./data/biological/EB095/stim_triggers_img_stim.npy')      # (300,)

In [30]:
stim_triggers_img_stim.shape

(300,)

In [11]:
dff_run

array([[-0.6598973 , -0.6166973 , -0.65530443, ..., -0.11214162,
         0.04662038,  0.00363585],
       [-0.3896974 , -0.18176754, -0.1064564 , ...,  0.9388432 ,
         0.6407711 ,  0.698943  ],
       [-0.42538935, -0.17485873, -0.4136891 , ..., -0.46224406,
        -0.77286345, -0.41269392],
       ...,
       [ 0.10833173,  0.13835469, -0.3402042 , ..., -0.12500976,
        -0.27234876, -0.0389387 ],
       [-0.08861481,  0.00205525, -0.229356  , ...,  0.4956022 ,
        -0.07777516, -0.13160245],
       [-0.21916537, -0.12675968, -0.24422294, ..., -0.16883187,
        -0.39703175, -0.05783821]], dtype=float32)